In [260]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException, NoSuchElementException

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

from time import sleep

In [261]:
PATH = r'C:\Program Files\Edge Driver\msedgedriver.exe'

service = Service(executable_path=PATH)
driver = webdriver.Edge(service=service)

driver.get("https://www.jetcost.ie/")
sleep(5)

In [262]:
def by_xpath(path) :
    global driver
    return driver.find_element(By.XPATH, path)

def xpath_click(path) :
    by_xpath(path).click()

In [263]:
xpath_click('//*[@id="iubenda-cs-banner"]/div/div/div/div[3]/div[2]/button[1]') # Deny cookies
xpath_click('//*[@id="base-search-form"]/div/div/div[2]/div/form/div[2]/div[1]/div/label[2]/span') # one way
xpath_click('//*[@id="base-search-form"]/div/div/div[2]/div/form/div[2]/div[2]/div[1]/div[1]/div[1]/div/div[2]/div[1]') # departure
typing1 = by_xpath('//*[@id="originFrom"]')
typing1.send_keys('casa')
sleep(1)
xpath_click('//*[@id="base-search-form"]/div/div/div[2]/div/form/div[2]/div[2]/div[1]/div[1]/div[1]/div/div[3]/ul/li/div') # casablanca
xpath_click('//*[@id="base-search-form"]/div/div/div[2]/div/form/div[2]/div[2]/div[1]/div[1]/div[3]/div/div[2]/div[1]') # destination
xpath_click('//*[@id="base-search-form"]/div/div/div[2]/div/form/div[2]/div[2]/div[1]/div[1]/div[3]/div/div[3]/ul/li/div/span[2]') # everywhere
xpath_click('//*[@id="base-search-form"]/div/div/div[2]/div/form/div[2]/div[2]/div[1]/div[2]/div[1]/div/div[2]/input') # date
xpath_click('//*[@id="base-search-form"]/div/div/div[2]/div/form/div[2]/div[2]/div[1]/div[2]/div[1]/div/div[3]/div[1]/div[1]/div[3]/div[5]/div[5]/div') # day
xpath_click('//*[@id="base-search-form"]/div/div/div[2]/div/form/div[2]/div[1]/div/label[2]/span') # !return!
xpath_click('//*[@id="base-search-form"]/div/div/div[2]/div/form/div[2]/div[2]/div[2]/div/div[1]/button[1]') # search
sleep(2)

In [264]:
data = {'country': [],
        'city': [],
        'duration': [],
        'trajectory': [],
        'departure_time': [],
        'arrival_time': [],
        'airport_id': [],
        'airline': [],
        'price': []}

In [265]:
countries = driver.find_elements(By.XPATH, "//span[@class='text-lg font-bold text-primary dark:text-primary-300 lg:px-2']")

for country in countries[208:] :
    country_name = country.text
    print(f'STARTING {country_name}')
    country.click()
    sleep(4)
    
    cities = driver.find_elements(By.XPATH, "//div[@class='text-black dark:text-neutral-200 tedark:text-neutral-200xt-lg lg:text-md font-bold']")
    
    for city in cities:
        city_name = city.text.strip()
        print(f'------STARTING {city_name}')
        city.click()
        sleep(6)
        
        driver.switch_to.window(driver.window_handles[1])
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        
        a = soup.find_all("div", class_="flex items-center flex-col md:flex-row flex-wrap")
        
        for d in a :
            
            duration = d.find("span", class_="text-sm block text-gray-600 dark:text-neutral-100 truncate mb-1")
            if duration is not None:
                data['duration'].append(duration.text)
            else:
                data['duration'].append(np.nan)
            
            price = d.find("span", class_="inline-block font-bold md:text-black md:dark:text-neutral-200 text-xl md:text-3xl md:mb-2 whitespace-nowrap")
            if price is not None:
                data['price'].append(price.text.replace('\n', '').strip())
            else:
                data['price'].append(np.nan)

            data['trajectory'].append(d.find_all("span")[5].text)
            
            departure_time = d.find("span", class_="font-bold sm:ml-1 lg:ml-2 md:text-md lg:text-lg xl:text-base")
            if departure_time is not None:
                data['departure_time'].append(departure_time.text)
            else:
                data['departure_time'].append(np.nan)
            
            arrival_time = d.find("span", class_="font-bold sm:mr-1 lg:mr-2 md:text-md lg:text-lg xl:text-base")
            if arrival_time is not None:
                data['arrival_time'].append(arrival_time.text)
            else:
                data['arrival_time'].append(np.nan)
            
            airport_id = d.find_all("span", class_="text-gray-600 dark:text-neutral-100 inline-block md:mb-px lg:mb-0")
            if (airport_id is not None) and (len(airport_id)>1) :
                data['airport_id'].append(airport_id[1].text.replace('\n', '').strip())
            else:
                data['airport_id'].append(np.nan)
            
            airline = d.find('img')
            if airline is not None:
                data['airline'].append(airline['src'])
            else:
                data['airline'].append(np.nan)
        
        data['city'] += [city_name]*len(a)
        data['country'] += [country_name]*len(a)
        
        for k in data.keys() : 
            print(len(data[k]), end=' ')
        print()
        
        print(f'------ENDING {city_name}')
        
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
    
    country.click()
    print(f'ENDING {country_name}')
    sleep(2)

print('END')
#322

STARTING Yemen
------STARTING Sanaa
20 20 20 20 20 20 20 20 20 
------ENDING Sanaa
ENDING Yemen
STARTING Zambia
------STARTING Livingstone
40 40 40 40 40 40 40 40 40 
------ENDING Livingstone
ENDING Zambia
STARTING Zimbabwe
------STARTING Harare
60 60 60 60 60 60 60 60 60 
------ENDING Harare
ENDING Zimbabwe
END


In [266]:
driver.quit()

In [267]:
df = pd.DataFrame(data)
#df.drop(df.index[:20], inplace=True)
#df.drop(df.index[-80:], inplace=True)
df['country'].value_counts() #7

country
Yemen       20
Zambia      20
Zimbabwe    20
Name: count, dtype: int64

In [268]:
df.to_csv('yfinale.csv')

In [269]:
647/20

32.35